# Enunciado

Se diseñará un intercambiador de calor para calentar agua mediante el uso de condensado a 67°C y 0,2 bar, que fluirá en el lado de la carcasa con un caudal másico de 50000 kg/h. Ei calor se transferirá a 30000 kg/h de agua procedente de acueducto a 17°C. Se recomienda una sola carcasa y un solo paso por tubo. Se sugiere una resistencia al ensuciamiento de 0,000176 m2
• K/W y el sobre-diseño de superficie no debe ser superior al 35%.
Se requiere una longitud máxima de tubo de 5 m debido a limitaciones de espacio. El material del tubo es acero al carbono (k = 60 W/m • K). El agua fluirá dentro de tubos 3/4 in (19 mm de diámetro exterior con 16 mm ID). La caída de presión máxima permitida en la carcasa es de 5,0 psi. La temperatura de salida del agua no debe ser inferior a 40°C.

In [292]:
caudal_masico_agua_condensada         = 50_000 / 3600 # kg/s
temperatura_entrada_agua_condensada   = 67 # ºC
presion_agua_condensada               = 0.2 # bar

caudal_transferido                    = 30_000 / 3600 # kg/s
temperatura_entrada                   = 17 # ºC
factor_ensuciamiento                  = 0.000176 # m^2K/W

longitud_maxima                       = 5 # m
k_material                            = 60 # W/(m * K)
diametro_tubos                        = 3/4 # in
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m

temperatura_salida                    = 40 #ºC
N_P = 2

$$ \dot{Q} = (\dot{m} *C_{p})_{c} * (T_{c2} -T_{c1})$$

In [238]:
from pyfluids import Fluid, FluidsList, Input
from core import ThermoFormules
import numpy as np

In [239]:
water = Fluid(FluidsList.Water)
water.name

Water

In [240]:
presion = 101325 * presion_agua_condensada # Pa

In [241]:
temperatura_promedio = (temperatura_entrada + temperatura_salida) /2
temperatura_promedio

28.5

In [242]:
state = water.with_state(
    Input.pressure(presion),
    Input.temperature(temperatura_promedio),
)
state.as_dict()

{'compressibility': 0.00014614052125805355,
 'conductivity': 0.6120463454734684,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.0564055844422,
 'dynamic_viscosity': 0.0008233612267610441,
 'enthalpy': 119478.276557885,
 'entropy': 416.01479691577845,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 119457.93132466005,
 'kinematic_viscosity': 8.266210850558526e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.623700026792162,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1505.4719747453637,
 'specific_heat': 4180.382725790069,
 'specific_volume': 0.0010039592079258241,
 'surface_tension': None,
 'temperature': 28.5,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [243]:
cp = state.specific_heat
cp

4180.382725790069

In [244]:
calor = caudal_transferido * cp * (temperatura_salida - temperatura_entrada)
calor

801240.0224430967

$$ T_{c2} = - \frac{Q}{(\dot{m} *C_{p})_{c}} + T_{c1}$$

In [245]:
state_cond = water.with_state(
    Input.pressure(presion),
    Input.temperature(40),
)
state_cond.as_dict()

{'compressibility': 0.00014132360145080458,
 'conductivity': 0.6284426249627434,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 992.1807698912999,
 'dynamic_viscosity': 0.000652718564006271,
 'enthalpy': 167544.45024584336,
 'entropy': 572.3967328979538,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 167524.02554033775,
 'kinematic_viscosity': 6.578625426068082e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 4.341067796088867,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1528.762927297536,
 'specific_heat': 4179.614601690888,
 'specific_volume': 0.001007880852306336,
 'surface_tension': None,
 'temperature': 40.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [246]:
cp_condensado = state_cond.specific_heat
cp_condensado

4179.614601690888

In [247]:
temperatura_salida_agua_condensada = temperatura_entrada_agua_condensada - calor/(caudal_masico_agua_condensada* cp_condensado)
temperatura_salida_agua_condensada

53.19746385406817

Asuma los coeficiente de trasferencia de calor

$$ \frac{1}{U_f} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + R_{ft} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

In [248]:
h_0 = 5_000 # W/m2k
h_1 = 4_000 # W/m2k

In [249]:
U_f = (
    1/h_0 + \
    diametro_exterior/diametro_interior*1/h_1 + \
    factor_ensuciamiento + \
    diametro_exterior/2 * np.log(diametro_exterior/diametro_interior)/k_material
) ** -1
U_f

1428.3987473789696

In [250]:
U_cond = (
    1/h_0 + \
    diametro_exterior/diametro_interior*1/h_1 + \
    diametro_exterior/2 * np.log(diametro_exterior/diametro_interior)/k_material
) ** -1
U_cond

1908.0887974581278

In [251]:
diferencia_temperatura_1 = temperatura_entrada_agua_condensada - temperatura_salida
diferencia_temperatura_2 = temperatura_salida_agua_condensada - temperatura_entrada
print(diferencia_temperatura_1, diferencia_temperatura_2)

diferencia_log_temperatura = (diferencia_temperatura_2 - diferencia_temperatura_1)/ np.log(diferencia_temperatura_2/diferencia_temperatura_1)
print(diferencia_log_temperatura)

27 36.19746385406817
31.374365034654137


Asume the F=0.9

In [252]:
F = 0.9
diferencia_log_temperatura_corregida =  F * diferencia_log_temperatura

Estimar las areas requeridas

In [253]:
A_f = calor / (U_f * diferencia_log_temperatura_corregida)
A_c = calor / (U_cond * diferencia_log_temperatura_corregida)

print(A_f, A_c)

19.865327039269854 14.871219985656523


In [254]:
area_ratio =  A_f/A_c - 1
area_ratio

0.3358236283526308

In [255]:
Cl         = 1
L          = 3    # m
CTP        = 0.93 # One tube pass
PitchRatio = 1.25

In [256]:
diametro_carcasa = 0.637 * (Cl/CTP * A_f*pow(PitchRatio,2)* diametro_exterior/L)**(1/2) 
print(diametro_carcasa)
diametro_carcasa = 0.3
print(diametro_carcasa)

0.29286805553111817
0.3


In [257]:
numeros_tubos = 0.785 * (CTP/Cl) * (diametro_carcasa)**2 /((PitchRatio * diametro_exterior)**2)
print(numeros_tubos)

116.48443213296399


Espacio entre deflectors entre 0.4 y 0.6 del diámetro de la carcasa

In [258]:
B = 0.6 * diametro_carcasa
B

0.18

In [259]:
B=0.2

### Método de kern

In [260]:
numeros_tubos = 124

In [261]:
T_b = (temperatura_entrada_agua_condensada + temperatura_salida_agua_condensada) / 2
print(T_b)

60.09873192703408


In [262]:
state_cond_kern = water.with_state(
    Input.pressure(presion),
    Input.temperature(T_b),
)
state_cond_kern.as_dict()

{'compressibility': 0.00013402552103725096,
 'conductivity': 0.6510520857299109,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 983.1095320685781,
 'dynamic_viscosity': 0.00046532065914316076,
 'enthalpy': 251593.82785635936,
 'entropy': 832.5330610116514,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 251573.21469035625,
 'kinematic_viscosity': 4.7331517390953516e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 2.9912339162674035,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1550.8817942419353,
 'specific_heat': 4185.176483842274,
 'specific_volume': 0.0010171806572721173,
 'surface_tension': None,
 'temperature': 60.098731927034066,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercen

In [263]:
densidad_cond_kern            = state_cond_kern.density
cp_cond_kern                  = state_cond_kern.specific_heat
viscosidad_dinamica_cond_kern = state_cond_kern.dynamic_viscosity
k_cond_kern                   = state_cond_kern.conductivity
prandtl_cond_kern             = state_cond_kern.prandtl

print(densidad_cond_kern, cp_cond_kern, viscosidad_dinamica_cond_kern, k_cond_kern, prandtl_cond_kern)

983.1095320685781 4185.176483842274 0.00046532065914316076 0.6510520857299109 2.9912339162674035


In [264]:
state_agua_kern = water.with_state(
    Input.pressure(presion),
    Input.temperature(28.5),
)
state_agua_kern.as_dict()

{'compressibility': 0.00014614052125805355,
 'conductivity': 0.6120463454734684,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.0564055844422,
 'dynamic_viscosity': 0.0008233612267610441,
 'enthalpy': 119478.276557885,
 'entropy': 416.01479691577845,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 119457.93132466005,
 'kinematic_viscosity': 8.266210850558526e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.623700026792162,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1505.4719747453637,
 'specific_heat': 4180.382725790069,
 'specific_volume': 0.0010039592079258241,
 'surface_tension': None,
 'temperature': 28.5,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [265]:
densidad_agua_kern            = state_agua_kern.density
cp_agua_kern                  = state_agua_kern.specific_heat
viscosidad_dinamica_agua_kern = state_agua_kern.dynamic_viscosity
k_agua_kern                   = state_agua_kern.conductivity
prandtl_agua_kern             = state_agua_kern.prandtl

print(densidad_agua_kern, cp_agua_kern, viscosidad_dinamica_agua_kern, k_agua_kern, prandtl_agua_kern)

996.0564055844422 4180.382725790069 0.0008233612267610441 0.6120463454734684 5.623700026792162


In [266]:
Pitch_Size              = 0.0254 # m

In [267]:
D_e = 4*(Pitch_Size**2 - np.pi*diametro_exterior**2/4)/(np.pi*diametro_exterior)
D_e

0.024233853930596717

In [268]:
C = Pitch_Size - diametro_exterior
print(C)

0.0063999999999999994


In [269]:
A_s = (0.39 * C * B) / Pitch_Size

print(A_s)

0.019653543307086616


In [270]:
G_s = caudal_masico_agua_condensada / A_s

print(G_s)

706.6862535612536


In [271]:
Re = G_s * D_e / viscosidad_dinamica_cond_kern

print('Reynolds: ', Re)

Reynolds:  36804.15022857419


In [272]:
T_w = 1/2 * (
    (temperatura_entrada + temperatura_salida)/2 +
    (temperatura_entrada_agua_condensada + temperatura_salida_agua_condensada)/2
)
T_w

44.29936596351704

In [273]:
state_average = water.with_state(
    Input.pressure(presion),
    Input.temperature(T_w),
)
state_average.as_dict()

{'compressibility': 0.00013965053088471624,
 'conductivity': 0.6338865642372711,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 990.4689406577198,
 'dynamic_viscosity': 0.0006032382945597438,
 'enthalpy': 185515.27025213066,
 'entropy': 629.3936046180794,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 185494.8102466364,
 'kinematic_viscosity': 6.09043120685e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 3.97809146126502,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1535.3339481498904,
 'specific_heat': 4180.203331493186,
 'specific_volume': 0.0010096227745778187,
 'surface_tension': None,
 'temperature': 44.29936596351706,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [274]:
viscodiad_dinamica_promedio_kern = state_average.dynamic_viscosity
viscodiad_dinamica_promedio_kern

0.0006032382945597438

In [275]:
h_0 = 0.36 * (D_e * G_s / viscosidad_dinamica_cond_kern)**0.55  * (cp_cond_kern* viscosidad_dinamica_cond_kern / k_cond_kern)**(1/3) * (viscosidad_dinamica_cond_kern / viscodiad_dinamica_promedio_kern)**0.14 * k_cond_kern / D_e
h_0

4360.865168079543

In [276]:
A_tp = np.pi * diametro_interior**2/4 * numeros_tubos/2 
A_tp

0.012465839649444298

In [277]:
u_m = caudal_transferido / (densidad_agua_kern * A_tp)
print(u_m)

0.671140249512848


In [278]:
Re = densidad_agua_kern * u_m * diametro_interior / viscosidad_dinamica_agua_kern
Re

12990.527566182289

## Turbulento

In [279]:
f = (1.58*np.log(Re) - 3.28)**-2
f

0.007322998176290353

In [280]:
Nu_b = (f/2)*(Re - 1000)*prandtl_agua_kern/(1 + 12.7*(f/2)**(1/2)*(prandtl_agua_kern**(2/3) - 1))
Nu_b

92.75808228403831

In [281]:
h_i = Nu_b * k_agua_kern/diametro_interior
print(h_i)

3548.2653296920575


In [282]:
U_f = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior / diametro_interior * factor_ensuciamiento + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    factor_ensuciamiento +
    1/h_i
) ** -1
U_f

972.0931001391128

In [283]:
U_c = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    1 / h_i
) ** -1
U_c

1553.4992858096562

In [284]:
f = np.e**(0.576 - 0.19*np.log(Re))
print(f)

N_b = longitud_maxima / B - 1 
print(N_b)

sigma = (viscosidad_dinamica_cond_kern / viscodiad_dinamica_promedio_kern)**0.14
print(sigma)

diferencia_presion = f * G_s ** 2 * (N_b + 1) * 0.39 / (sigma * 2 * densidad_agua_kern * D_e)
diferencia_presion

0.2941470733613788
24.0
0.9643104660521746


30765.865692158754

In [285]:
diferenceia_presion_psi =  diferencia_presion / 6895
diferenceia_presion_psi

4.462054487622734

In [286]:
F = 0.95
diferencia_log_temperatura_corregida =  F * diferencia_log_temperatura
diferencia_log_temperatura_corregida

29.80564678292143

In [287]:
A_of = calor / (U_f * diferencia_log_temperatura_corregida)
A_of

27.653889518450132

In [291]:
L = A_of / (np.pi * diametro_exterior * numeros_tubos)

print(L)
L = round(L)
print(L)

3.7362081600836095
4


In [293]:
diferencia_presion = (4 * f * L * N_P / diametro_interior + 4 * N_P) * densidad_agua_kern * (0.67)**2/2
diferencia_presion

133310.41757008457